In [1]:
# Palakad coordinates
pal_lat, pal_lon = 10.7867, 76.6548

# Bangalore coordinates
blr_lat, blr_lon = 12.9716, 77.5946

# Thiruvananthapuram coordinates
tvm_lat, tvm_lon = 8.50606, 76.96153

# Kollam coordinates
kol_lat, kol_lon = 8.88795, 76.59550

# Kochi coordinates
cok_lat, cok_lon = 9.9312, 76.2673

In [2]:
from geopy.geocoders import Nominatim

In [3]:
# fetch osm id
geolocator = Nominatim(user_agent="sample_app")
loc_osmid = "{lat}, {lon}".format(lat=blr_lat, lon=blr_lon)
location = geolocator.reverse(loc_osmid)
location.raw.get('osm_id')

38773287

In [4]:
# osmid for each coordinates 
tvm_osmid, kol_osmid = 955820326,281828280
blr_osmid, pal_osmid = 38773287, 464139803

In [5]:
import networkx as nx

In [6]:
# initialize multidigraph
G = nx.MultiDiGraph(crs="EPSG:4326")

# add nodes
G.add_nodes_from([tvm_osmid, blr_osmid, pal_osmid, kol_osmid])
# set x and y coordinates 
G.nodes[tvm_osmid].update({'x': tvm_lon, 'y': tvm_lat})
G.nodes[kol_osmid].update({'x': kol_lon, 'y': kol_lat})
G.nodes[pal_osmid].update({'x': pal_lon, 'y': pal_lat})
G.nodes[blr_osmid].update({'x': blr_lon, 'y': blr_lat})

# add edges 
G.add_edges_from([(tvm_osmid, blr_osmid), (kol_osmid, pal_osmid)])

[0, 0]

In [7]:
import osmnx as ox

In [8]:
# convert multidigraph nodes to geodataframe
gdf_nodes = ox.utils_graph.graph_to_gdfs(
  G, nodes=True, edges=False, 
  node_geometry=True, fill_edge_geometry=False)

# convert multidigraph edges to geodataframe
gdf_edges = ox.utils_graph.graph_to_gdfs(
  G, nodes=False, edges=True, 
  node_geometry=False, fill_edge_geometry=True)

# apply projection
gdf_nodes['geometry'] = gdf_nodes.geometry.to_crs("EPSG:7781")
# update x and y coordinates with projected coordinates
gdf_nodes['x'] = gdf_nodes['geometry'].x 
gdf_nodes['y'] = gdf_nodes['geometry'].y

# convert edges to projected coordinates
gdf_edges['geometry'] = gdf_edges.geometry.to_crs("EPSG:7781")


In [9]:
gdf_nodes

,x,y,geometry
osmid,,,
955820326,1.105863e+06,7.796105e+05,POINT (1105863.471 779610.506)
38773287,1.173015e+06,1.273927e+06,POINT (1173015.423 1273927.376)
464139803,1.071608e+06,1.031786e+06,POINT (1071608.015 1031786.480)
281828280,1.065496e+06,8.217651e+05,POINT (1065495.736 821765.127)


In [10]:
gdf_edges

,,,geometry
u,v,key,
955820326,38773287,0,"LINESTRING (1105863.471 779610.506, 1173015.42..."
281828280,464139803,0,"LINESTRING (1065495.736 821765.127, 1071608.01..."


In [11]:
gdf_edges.explore()

In [12]:
# set crs attribute
graph_attrs = {"crs": "EPSG:7781"}

# convert geodataframe to multidigraph
proj_multidigraph = ox.utils_graph.graph_from_gdfs(
  gdf_nodes, gdf_edges, graph_attrs=graph_attrs)

In [13]:
import geopandas as gpd
from shapely import Point

In [14]:
# create geodataframe for x-y coordinate
cok_node_dict = {'col1':['Kochi'], 'geometry': [Point(cok_lon, cok_lat)]}
cok_node_gdf = gpd.GeoDataFrame(cok_node_dict, crs="EPSG:4326")

# convert to projected coordinates
cok_node_gdf['geometry'] = cok_node_gdf.geometry.to_crs("EPSG:7781")
cok_proj_lon = cok_node_gdf['geometry'].x[0]
cok_proj_lat =  cok_node_gdf['geometry'].y[0]

In [15]:
ox.distance.nearest_edges(
  proj_multidigraph, cok_proj_lon, cok_proj_lat, 
  interpolate=None, return_dist=True)

((281828280, 464139803, 0), 39525.24413890172)